<hr style="border:2px solid #0281c9"> </hr>

<img align="left" alt="ESO Logo" src="http://archive.eso.org/i/esologo.png">  

<div align="center">
  <h1 style="color: #0281c9; font-weight: bold;">ESO Science Archive</h1> 
  <h2 style="color: #0281c9; font-weight: bold;">Jupyter Notebooks</h2>
</div>

<hr style="border:2px solid #0281c9"> </hr>

## Query by source name/position

This notebook demonstrates how to use the ``astroquery.eso`` module to search for and download science-ready **HAWK-I** near-infrared data from the ESO archive, specifically targeting **Sgr A**, the supermassive black hole at the center of the Milky Way. It guides users through constructing and executing the query to locate relevant **HAWK-I** observations, filtering results to identify processed data products, and downloading these datasets for further analysis, providing a streamlined approach to accessing ESO’s extensive observational resources programatically. 

<hr style="border:2px solid #0281c9"> </hr>

### Importing and basic usage of astroquery.eso 

In this example, we will create an instance of the ESO class using:

```python
from astroquery.eso import Eso # import the ESO module from astroquery
eso = Eso()  # create an instance of the ESO class
```

This instance will be used to demonstrate various queries to the ESO Archive, including searching for raw data and data products from a range of instruments and sources.

In [1]:
from astroquery.eso import Eso # import the ESO module from astroquery

In [2]:
eso = Eso() # create an instance of the ESO class 

### Identify available collections (reduced data)

ESO provides access to processed or reduced data, which has undergone various calibration and analysis steps to make it more immediately useful for scientific research. These processed datasets are grouped into `collections`, each corresponding to a specific survey, instrument, or science case. The list of available collections can be obtained as follows:

In [8]:
collections = eso.list_collections() # list all collections
collections[:3] # print the first 3 collections

['081.C-0827', '092.A-0472', '096.B-0054']

In [9]:
'HAWKI' in collections # check if 'HAWKI' is in the collections

True

### Performing a Small Cone Search Around Sgr A*

To perform a small cone search of **10 arcseconds** around the source **Sgr A**, we first need to resolve the source name to obtain its **Right Ascension (RA)** and **Declination (Dec)** coordinates. This can be done using an online name resolver such as the ``SkyCoord.from_name()`` method from astropy.coordinates, which allows us to resolve object names into precise RA/Dec coordinates using online astronomical databases. This gives: 

```python
ra = 266.835
dec = -28.38528
```

In coordinate values of degrees in the ICRS (International Celestial Reference System) frame.

In [10]:
from astropy.coordinates import SkyCoord # import the SkyCoord class from the astropy.coordinates module
import astropy.units as u # import the astropy.units module

coords = SkyCoord.from_name('Sgr A*') # create a SkyCoord object from the name of the source
radius = 20 *u.arcmin # set the radius of the search to 20 arcminutes

We use `astroquery.eso` to search for observations of **Sgr A** taken with the **HAWKI** instrument on the **VLT (Very Large Telescope)**.

This search can also be accessed via the following: 

[tap-link]: https://archive.eso.org/tap_obs/sync?REQUEST=doQuery&amp;LANG=ADQL&amp;MAXREC=200&amp;FORMAT=txt&amp;QUERY=SELECT%20*%0Afrom%20ivoa.obscore%0Awhere%20intersects(s_region,%20circle(%27%27,%20266.835,%20-28.38528,%200.1666))=1%0AAND%20instrument_name%20=%20%27HAWKI%27

[sp-link]: https://archive.eso.org/scienceportal/home?data_release_date=*:2025-02-07&pos=266.41682,-29.00782&r=0.016667&fovcorners=266.800868,-29.226927,266.033328,-29.226929,266.034957,-28.787142,266.799236,-28.787141&ins_id=HAWKI&dp_type=IMAGE&sort=dist,-fov,-obs_date&s=P%2fDSS2%2fcolor&f=0.669626&fc=266.799236,-28.787141&cs=J2000&av=true&ac=false&c=9,10,11,12,13,15,16,17,18,19,20,21&ta=RES&dts=true&sdtm=%7b%22IMAGE%22%3atrue%7d&at=266.41682,-29.00782&sr=i

- **TAP** - [see here][tap-link]
- **ESO Science Portal** [see here](sp-link)

In [11]:
instrument = 'HAWKI'       # set the instrument to HAWKI
ra = coords.ra             # get the right ascension of the source
dec = coords.dec           # get the declination of the source
radius = radius.to('deg')  # convert the radius to degrees

table = eso.query_collections(instrument, ra=ra.value, dec=dec.value, radius=radius.value) # query the ESO archive for VIRCAM data around Sgr A*

We can now inspect the results, which are stored as an ``astropy.table``. However note here these results include all data from the science achive, included non-science data... 

In [13]:
table[0] # print the first row of the table

abmaglim,access_estsize,access_format,access_url,bib_reference,calib_level,dataproduct_subtype,dataproduct_type,dp_id,em_max,em_min,em_res_power,em_xel,facility_name,filter,gal_lat,gal_lon,instrument_name,last_mod_date,multi_ob,n_obs,o_calib_status,o_ucd,obs_collection,obs_creator_did,obs_creator_name,obs_id,obs_publisher_did,obs_release_date,obs_title,obstech,p3orig,pol_states,pol_xel,preview_html,proposal_id,publication_date,release_description,s_dec,s_fov,s_pixel_scale,s_ra,s_region,s_resolution,s_xel1,s_xel2,snr,strehl,t_exptime,t_max,t_min,t_resolution,t_xel,target_name
mag,kB,,,,,,,,m,m,,,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,deg,deg,arcsec,deg,,arcsec,,,,,s,d,d,s,,
float64,int64,object,object,object,int32,object,object,object,float64,float64,float64,int64,object,object,float64,float64,object,object,object,int32,object,object,object,object,object,object,object,object,object,object,object,object,int64,object,object,object,object,float64,float64,float64,float64,object,float64,int64,int64,float64,float64,float64,float64,float64,float64,int64,object
21.069,25182,application/x-votable+xml;content=datalink,http://archive.eso.org/datalink/links?ID=ivo://eso.org/ID?ADP.2019-08-19T15:05:20.895,,2,srctbl,measurements,ADP.2019-08-19T15:05:20.895,2.307e-06,1.982e-06,6.5985,1,ESO-VLT-U4,Ks,-0.001327,0.258407,HAWKI,2020-10-27T08:54:19.823Z,S,1,absolute,,HAWKI,ivo://eso.org/origfile?HI_SOBJ_2355202_2019-07-12T06:47:40.886_f1Ks_f2OPEN_AO.fits,"ZOCCALI, MANUELA",2355202,ivo://eso.org/ID?ADP.2019-08-19T15:05:20.895,2020-07-12T07:00:41.643Z,,IMAGE,IDP,,--,https://archive.eso.org/dataset/ADP.2019-08-19T15:05:20.895,0103.B-0262(A),2019-08-19T15:12:41Z,http://www.eso.org/rm/api/v1/public/releaseDescriptions/87,-28.716261,0.18998676777,--,266.55974,UNION J2000 (POLYGON J2000 266.66529 -28.694905 266.623308 -28.753569 266.55647799999997 -28.716751 266.598483 -28.658108 POLYGON J2000 266.625682 -28.750113 266.583623 -28.808703 266.516814 -28.771821 266.558896 -28.713251 POLYGON J2000 266.60258799999997 -28.660628 266.560609 -28.719283 266.493811 -28.682455 266.535813 -28.623821 POLYGON J2000 266.563044 -28.715763 266.52104099999997 -28.774408 266.454191 -28.737563 266.496216 -28.678939),0.327,--,--,--,--,160.0,58676.28740017,58676.28311211,370.488384,--,NPL054


We can check all the column names in the table by printing the ``table.colnames`` attribute to see what else we could filter... 

In [15]:
table[0].colnames # print the column names of the table

['abmaglim',
 'access_estsize',
 'access_format',
 'access_url',
 'bib_reference',
 'calib_level',
 'dataproduct_subtype',
 'dataproduct_type',
 'dp_id',
 'em_max',
 'em_min',
 'em_res_power',
 'em_xel',
 'facility_name',
 'filter',
 'gal_lat',
 'gal_lon',
 'instrument_name',
 'last_mod_date',
 'multi_ob',
 'n_obs',
 'o_calib_status',
 'o_ucd',
 'obs_collection',
 'obs_creator_did',
 'obs_creator_name',
 'obs_id',
 'obs_publisher_did',
 'obs_release_date',
 'obs_title',
 'obstech',
 'p3orig',
 'pol_states',
 'pol_xel',
 'preview_html',
 'proposal_id',
 'publication_date',
 'release_description',
 's_dec',
 's_fov',
 's_pixel_scale',
 's_ra',
 's_region',
 's_resolution',
 's_xel1',
 's_xel2',
 'snr',
 'strehl',
 't_exptime',
 't_max',
 't_min',
 't_resolution',
 't_xel',
 'target_name']

## Downloading datasets from the archive

The returned table has an ``dp_id`` column. It can be used to retrieve the datasets with ``retrieve_data()``.

The file names, returned in ``data_files``, points to the decompressed datasets (without the .Z extension) that have been locally downloaded. 

The default location (in the ``astropy`` cache) of the decompressed datasets can be adjusted by providing a destination keyword in the call to ``retrieve_data()``. By default, if a requested dataset is already found, it is not downloaded again from the archive. To force the retrieval of data that are present in the destination directory, use ``continuation=True`` in the call to ``retrieve_data()``.

In [16]:
data_files = eso.retrieve_data(table['dp_id'][0]) # download the first data product
# data_files = eso.retrieve_data(table['dp_id'][0], continuation=True) # force the download of all files even if present in the destination directory
# data_files = eso.retrieve_data(table['dp_id'][0], destination='./') # specify the destination directory for the download

INFO: Downloading datasets ... [astroquery.eso.core]
INFO: Downloading 1 files ... [astroquery.eso.core]
INFO: Downloading file 1/1 https://dataportal.eso.org/dataPortal/file/ADP.2019-08-19T15:05:20.895 to /Users/abarnes/.astropy/cache/astroquery/Eso [astroquery.eso.core]
INFO: Successfully downloaded dataset ADP.2019-08-19T15:05:20.895 to ... [astroquery.eso.core]
INFO: Done! [astroquery.eso.core]


<hr style="border:2px solid #0281c9"> </hr>